In [ ]:
import ee
import geopandas as gpd
import pandas as pd
import pandas as pd
import json
import configparser
import time

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
config = configparser.ConfigParser()
config.read(r"./config.ini")
start_yy = int(config.get("myvars", "start_yy"))
end_yy = int(config.get("myvars", "end_yy"))+1
start_mm = int(config.get("myvars", "start_mm"))
end_mm = int(config.get("myvars", "end_mm"))+1

ee_collection = config.get("myvars", "ee_col")
path = config.get("myvars", "path")
band_ee = config.get("myvars", "band_ee")

folder = config.get("myvars", "folder")

In [ ]:
data = ee.ImageCollection(ee_collection);

data_band = data.select(band_ee)

for yy in range(start_yy,end_yy):
    yy_count = 1
    
    for mm in range (start_mm,end_mm):
        if(mm==2):
            day_l = 28
        else :
            day_l = 30
        
        ##for dd in range(1:day_l+1):
            
    
        s_date = str(yy) + '-' + str(mm) + '-1'
        e_date = str(yy) + '-' + str(mm) + '-' + str(day_l)

        dataset = data_band.filterDate(s_date, e_date).mean()

        geom = gpd.read_file(path)

        js = json.loads(geom.to_json())
        shpz = ee.Geometry(ee.FeatureCollection(js).geometry())

        image = dataset.clip(shpz)
        
        veca = 'vec_'+str(mm)+"_"+str(yy)
        print(veca)
        
        task_config = {
          'image': image,
          'scale' : 250,
          'description' : veca,
          'fileNamePrefix': veca,
          'folder' : folder,
          'region': shpz,
          }
        print("Exporting")
        task=ee.batch.Export.image.toDrive(**task_config)
        task.start()